__import__ required Python libraries

In [1]:
from bs4 import BeautifulSoup
import datetime
import pandas as pd
import requests

__generate__ the hockey-reference url

In [2]:
stats_month = 2
stats_day = 9
stats_year = 2018
stats_webpage = 'https://www.hockey-reference.com/friv/dailyleaders.fcgi?month={}&day={}&year={}'.format(
                 stats_month, stats_day, stats_year)

__create__ the soup object

In [3]:
r = requests.get(stats_webpage)
html_doc = r.text
soup = BeautifulSoup(html_doc, 'html.parser')

__get__ all records from the skaters table

In [4]:
skaters_table = soup.find(id='skaters').find_all('td')
skaters_tbody = [cell.get_text() for cell in skaters_table]
skaters_records = [tuple(skaters_tbody[row: row + 28]) for row in range(0, len(skaters_tbody), 28)]

__make__ a list of player ids

In [5]:
player_id_list = []

for player_id in range(len(skaters_table)):
    try:
        player_id_list.append(skaters_table[player_id]['data-append-csv'])
    except:
        pass

__load__ the skaters data into a Pandas DataFrame

In [6]:
columns_list = ['Player Name','Position','Team','Home Away','Opponent','Result','Boxscore','G','A','PTS','Plus Minus','PIM','EV','PP','SH','GW','EV','PP','SH','S','S%','SHFT','TOI','HIT','BLK','FOW','FOL','FO%']
df_skaters = pd.DataFrame.from_records(skaters_records, columns=columns_list)
df_skaters.insert(0, 'Date', datetime.datetime(stats_year, stats_month, stats_day))

__concatenate__ the Player IDs to the skaters DataFrame

In [7]:
df_player_id = pd.DataFrame(player_id_list, columns=['Player ID'])
df_concat = pd.concat([df_player_id, df_skaters], axis=1)

__delete__ the *Boxscore* column and __replace__ the contents of the *Home Away* column

In [8]:
del df_concat['Boxscore']
df_concat['Home Away'] = df_concat['Home Away'].apply(lambda x: 'Home' if x == '' else 'Away') 

__index__ the skaters DataFrame by *Date* and *Player ID* and __sort__ by the index

In [9]:
df_concat.set_index(['Date','Player ID'],inplace=True, drop=True)
df_concat.sort_index(axis=0, inplace=True, ascending=True)

__result__ of the above code

In [10]:
print(df_concat)

                              Player Name Position Team Home Away Opponent  \
Date       Player ID                                                         
2018-02-09 abdelju01    Justin Abdelkader       LW  DET      Away      NYI   
           ahose01          Sebastian Aho       RW  CAR      Home      VAN   
           ahose02          Sebastian Aho        D  NYI      Home      DET   
           anderjo05        Josh Anderson       RW  CBJ      Away      WSH   
           andrean01        Andy Andreoff       LW  LAK      Away      FLA   
           archida02     Darren Archibald       RW  VAN      Away      CAR   
           armiajo01           Joel Armia       RW  WPG      Home      STL   
           astonza01  Zachary Aston-Reese        C  PIT      Away      DAL   
           athanan01   Andreas Athanasiou       LW  DET      Away      NYI   
           atkinca01         Cam Atkinson       RW  CBJ      Away      WSH   
           backlmi01      Mikael Backlund        C  CGY      Awa